In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sns
from dotenv import load_dotenv

In [ ]:
tf.config.threading.set_intra_op_parallelism_threads(8)

In [ ]:
load_dotenv()

In [ ]:
TRAIN_REAL_PATH = os.getenv("TRAIN_REAL_PATH")
TRAIN_FAKE_PATH = os.getenv("TRAIN_FAKE_PATH")
TEST_FAKE_PATH = os.getenv("TEST_FAKE_PATH")
TEST_REAL_PATH = os.getenv("TEST_REAL_PATH")

print("TRAIN_REAL_PATH:", TRAIN_REAL_PATH)
print("TRAIN_FAKE_PATH:", TRAIN_FAKE_PATH)
print("TEST_REAL_PATH:", TEST_REAL_PATH)
print("TEST_FAKE_PATH:", TEST_FAKE_PATH)


train_real_images = np.array([cv2.imread(TRAIN_REAL_PATH+'/'+file) for file in os.listdir(TRAIN_REAL_PATH)])
train_fake_images = np.array([cv2.imread(TRAIN_FAKE_PATH+'/'+file) for file in os.listdir(TRAIN_FAKE_PATH)])
test_fake_images = np.array([cv2.imread(TEST_REAL_PATH+'/'+file) for file in os.listdir(TEST_REAL_PATH)])
test_real_images = np.array([cv2.imread(TEST_FAKE_PATH+'/'+file) for file in os.listdir(TEST_FAKE_PATH)])

In [ ]:
train_real_images = train_real_images / 255
train_fake_images = train_fake_images / 255
test_real_images = test_real_images / 255
test_fake_images = test_fake_images / 255

In [ ]:
train_images = np.concatenate((train_real_images, train_fake_images), axis=0)
test_images = np.concatenate((test_real_images, test_fake_images), axis=0)
train_labels = np.concatenate((np.zeros(len(train_real_images)), np.ones(len(train_fake_images))), axis=0)
test_labels = np.concatenate((np.zeros(len(test_real_images)), np.ones(len(test_fake_images))), axis=0)

In [ ]:
train_shuffle = np.random.permutation(len(train_images))
test_shuffle = np.random.permutation(len(test_images))

train_images = train_images[train_shuffle]
train_labels = train_labels[train_shuffle]
test_images = test_images[test_shuffle]
test_labels = test_labels[test_shuffle]

In [ ]:
train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32,32,3), padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.1),
    tf.keras.layers.Dense(64, activation='relu', kernel_initializer='he_uniform'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=2,
    restore_best_weights=True
)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(
    x=train_images,
    y=train_labels,
    validation_data=(val_images, val_labels),
    epochs=100,
    callbacks=[early_stopping],
    verbose=1,
    batch_size=128
)

In [ ]:
print(model.summary())

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()

In [ ]:
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
y_pred = model.predict(test_images)
y_pred = np.round(y_pred)
cm = confusion_matrix(test_labels, y_pred)
sns.heatmap(cm, annot=True, fmt='d')
plt.show()